# AC 데이터 분석

이 노트북은 Clea 스토어의 AC 제어 데이터를 분석합니다.

## 데이터 로드 준비
데이터 경로와 로딩 함수를 설정합니다.


In [124]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료!")


라이브러리 임포트 완료!


In [125]:
# 데이터 경로 설정 (올바른 경로로 수정)
data_path = "../../Clea/ac-control/"
print(f"데이터 경로: {data_path}")

# 사용 가능한 파일 확인
if os.path.exists(data_path):
    csv_files = glob.glob(os.path.join(data_path, "*.csv"))
    print(f"CSV 파일 {len(csv_files)}개 발견")
    
    # 파일명 샘플 출력
    for i, file in enumerate(csv_files[:5]):
        print(f"  {i+1}. {os.path.basename(file)}")
    
    if len(csv_files) > 5:
        print(f"  ... 외 {len(csv_files)-5}개 파일")
else:
    print(f"경로가 존재하지 않습니다: {data_path}")
    print("현재 작업 디렉토리:", os.getcwd())
    print("상위 디렉토리 확인:")
    print("  - ../../Clea/ 존재:", os.path.exists("../../Clea/"))
    print("  - ../../Clea/ac-control/ 존재:", os.path.exists("../../Clea/ac-control/"))


데이터 경로: ../../Clea/ac-control/
CSV 파일 30개 발견
  1. ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
  2. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
  3. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
  4. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
  5. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
  ... 외 25개 파일


In [126]:
# 데이터 로딩 함수
def load_csv_data(file_path):
    """CSV 파일을 로드하는 함수"""
    try:
        df = pd.read_csv(file_path)
        print(f"파일 로드 성공: {os.path.basename(file_path)}")
        print(f"  - 행 수: {len(df)}")
        print(f"  - 열 수: {len(df.columns)}")
        print(f"  - 열명: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"파일 로딩 오류: {e}")
        return None

def list_csv_files(directory):
    """디렉토리 내의 모든 CSV 파일 목록 반환"""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    print(f"'{directory}' 디렉토리에서 {len(csv_files)}개의 CSV 파일 발견")
    return csv_files

print("데이터 로딩 함수 정의 완료!")


데이터 로딩 함수 정의 완료!


In [127]:
# 사용 예시 (올바른 경로로 수정)
# 특정 파일 로드
# df = load_csv_data("../../Clea/ac-control/ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv")

# 또는 파일 목록에서 선택
# csv_files = list_csv_files("../../Clea/ac-control/")
# df = load_csv_data(csv_files[0])  # 첫 번째 파일 로드

print("데이터 로드 준비 완료!")
print("위의 주석을 해제하고 원하는 데이터를 로드하세요.")
print("올바른 경로: ../../Clea/ac-control/")


데이터 로드 준비 완료!
위의 주석을 해제하고 원하는 데이터를 로드하세요.
올바른 경로: ../../Clea/ac-control/


## 모든 CSV 파일 연결 및 월별 분석

모든 AC 제어 데이터를 연결하여 월별 온도 분석을 수행합니다.


In [128]:
# 모든 CSV 파일 연결 및 월별 분석 (두 건물 처리)
def load_all_csv_files(directory):
    """디렉토리 내 모든 CSV 파일을 로드하고 연결"""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    print(f"총 {len(csv_files)}개의 CSV 파일 발견")
    
    all_data = []
    
    for i, file_path in enumerate(csv_files):
        print(f"로딩 중... ({i+1}/{len(csv_files)}) {os.path.basename(file_path)}")
        
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"  오류: {e}")
            continue
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\n통합 완료! 총 {len(combined_df)}개 행")
        return combined_df
    else:
        print("로딩된 데이터가 없습니다.")
        return None

def process_building_data(building_name, data_path):
    """특정 건물의 데이터를 처리"""
    print(f"\n{'='*60}")
    print(f"=== {building_name} 건물 데이터 처리 ===")
    print(f"{'='*60}")
    
    # 모든 데이터 로드
    print(f"=== {building_name} CSV 파일 로드 ===")
    all_data = load_all_csv_files(data_path)
    
    if all_data is None:
        print(f"{building_name} 데이터 로딩 실패")
        return None
    
    return all_data

# 두 건물 데이터 처리
buildings = {
    'Clea': '../../Clea/ac-control/',
    'IsetanMitsukoshi': '../../IsetanMitsukoshi/ac-control/'
}

building_data = {}

for building_name, data_path in buildings.items():
    print(f"\n{building_name} 건물 데이터 처리 시작...")
    data = process_building_data(building_name, data_path)
    if data is not None:
        building_data[building_name] = data
        print(f"{building_name} 데이터 처리 완료: {len(data)}개 행")
    else:
        print(f"{building_name} 데이터 처리 실패")

print(f"\n총 {len(building_data)}개 건물 데이터 처리 완료")



Clea 건물 데이터 처리 시작...

=== Clea 건물 데이터 처리 ===
=== Clea CSV 파일 로드 ===
총 30개의 CSV 파일 발견
로딩 중... (1/30) ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
로딩 중... (2/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
로딩 중... (3/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
로딩 중... (4/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
로딩 중... (5/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
로딩 중... (6/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv
로딩 중... (7/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv
로딩 중... (8/30) ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv
로딩 중... (9/30) ac-control-50,66,

In [129]:
# 데이터 구조 확인 및 날짜 컬럼 처리
if all_data is not None:
    print("=== 데이터 구조 확인 ===")
    print(f"데이터 형태: {all_data.shape}")
    print(f"컬럼 목록: {list(all_data.columns)}")
    
    # 날짜/시간 컬럼 찾기
    time_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    print(f"시간 관련 컬럼: {time_columns}")
    
    # 온도 관련 컬럼 찾기
    temp_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['temp', '온도', 'temperature'])]
    print(f"온도 관련 컬럼: {temp_columns}")
    
    # 샘플 데이터 확인
    print(f"\n샘플 데이터 (처음 3행):")
    print(all_data.head(3))
    
    # 각 컬럼의 데이터 타입 확인
    print(f"\n데이터 타입:")
    print(all_data.dtypes)


=== 데이터 구조 확인 ===
데이터 형태: (983682, 17)
컬럼 목록: ['A/C Name', 'Datetime', 'Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'A/C ON/OFF', 'A/C Mode', 'A/C Fan Speed', 'Naive Energy Level', 'Airux Energy Level', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.', 'source_file', 'year_month', 'year', 'month']
시간 관련 컬럼: ['Datetime']
온도 관련 컬럼: ['Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.']

샘플 데이터 (처음 3행):
   A/C Name                         Datetime  Outdoor Temp.  Indoor Temp.  \
3      G-22 2025-04-30 23:55:00.084567+09:00            6.0          18.0   
12     G-21 2025-04-30 23:40:00.023055+09:00            6.0          19.0   
18     G-21 2025-04-30 23:35:00.072182+09:00            6.0          20.0   

    A/C Set Temperature A/C ON/OFF A/C Mode A/C Fan Speed  Naive Energy Level  \
3                  22.0         ON     HEAT        Medium                 100   
12                 26.0         ON     C

In [130]:
# 날짜 컬럼 처리 및 월별 분석 (A/C ON 상태만 필터링)
def process_date_column(df, date_column):
    """날짜 컬럼을 처리하고 월 정보 추출"""
    if date_column not in df.columns:
        print(f"날짜 컬럼 '{date_column}'이 존재하지 않습니다.")
        return df
    
    # 날짜 컬럼을 datetime으로 변환
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    
    # 월 정보 추출
    df['year_month'] = df[date_column].dt.to_period('M')
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    
    return df

def analyze_monthly_temperature(df, indoor_temp_col, ac_set_temp_col):
    """월별 온도 분석 (A/C ON 상태만)"""
    if indoor_temp_col not in df.columns or ac_set_temp_col not in df.columns:
        print(f"필요한 온도 컬럼이 존재하지 않습니다.")
        print(f"Indoor Temp: {indoor_temp_col in df.columns}")
        print(f"A/C Set Temperature: {ac_set_temp_col in df.columns}")
        return None
    
    # 월별 그룹화 및 통계 계산
    monthly_stats = df.groupby('year_month').agg({
        indoor_temp_col: ['mean', 'std', 'count'],
        ac_set_temp_col: ['mean', 'std', 'count']
    }).round(2)
    
    # 컬럼명 정리
    monthly_stats.columns = [
        'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
        'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
    ]
    
    # 인덱스를 문자열로 변환
    monthly_stats.index = monthly_stats.index.astype(str)
    
    return monthly_stats

# 날짜 컬럼 처리 및 A/C ON 상태 필터링
if all_data is not None:
    time_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    
    if time_columns:
        date_column = time_columns[0]
        print(f"날짜 컬럼 '{date_column}' 사용")
        all_data = process_date_column(all_data, date_column)
        
        # A/C ON/OFF 컬럼 찾기 및 필터링
        ac_onoff_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ["A/C ON/OFF", 'on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
        print(f"A/C ON/OFF 관련 컬럼: {ac_onoff_columns}")
        
        if ac_onoff_columns:
            ac_onoff_col = ac_onoff_columns[0]
            print(f"A/C ON/OFF 컬럼 '{ac_onoff_col}' 사용")
            
            # 필터링 전 데이터 수
            total_rows = len(all_data)
            print(f"전체 데이터: {total_rows}개 행")
            
            # A/C ON 상태만 필터링
            ac_on_data = all_data[all_data[ac_onoff_col] == 'ON'].copy()
            ac_on_rows = len(ac_on_data)
            print(f"A/C ON 상태 데이터: {ac_on_rows}개 행 ({ac_on_rows/total_rows*100:.1f}%)")
            
            if ac_on_rows > 0:
                # 월별 분석 (A/C ON 상태만)
                indoor_temp_col = 'Indoor Temp.'
                ac_set_temp_col = 'A/C Set Temperature'
                
                monthly_analysis = analyze_monthly_temperature(ac_on_data, indoor_temp_col, ac_set_temp_col)
                
                if monthly_analysis is not None:
                    print("\n=== A/C ON 상태 월별 온도 분석 결과 ===")
                    print(monthly_analysis)
                    
                    # 필터링된 데이터를 all_data로 업데이트
                    all_data = ac_on_data
                else:
                    print("A/C ON 상태 데이터로 월별 분석을 수행할 수 없습니다.")
            else:
                print("A/C ON 상태 데이터가 없습니다.")
        else:
            print("A/C ON/OFF 컬럼을 찾을 수 없습니다. 전체 데이터로 분석을 진행합니다.")
            
            # 월별 분석 (전체 데이터)
            indoor_temp_col = 'Indoor Temp.'
            ac_set_temp_col = 'A/C Set Temperature'
            
            monthly_analysis = analyze_monthly_temperature(all_data, indoor_temp_col, ac_set_temp_col)
            
            if monthly_analysis is not None:
                print("\n=== 월별 온도 분석 결과 ===")
                print(monthly_analysis)
            else:
                print("월별 분석을 수행할 수 없습니다.")
    else:
        print("날짜/시간 컬럼을 찾을 수 없습니다.")


날짜 컬럼 'Datetime' 사용
A/C ON/OFF 관련 컬럼: ['A/C ON/OFF']
A/C ON/OFF 컬럼 'A/C ON/OFF' 사용
전체 데이터: 983682개 행
A/C ON 상태 데이터: 983682개 행 (100.0%)

=== A/C ON 상태 월별 온도 분석 결과 ===
            Indoor_Temp_Mean  Indoor_Temp_Std  Indoor_Temp_Count  \
year_month                                                         
2024-07                27.43             1.89              84927   
2024-08                26.83             2.15              87083   
2024-09                25.69             2.00              83463   
2024-10                25.47             2.07              52638   
2024-11                23.88             2.80              41735   
2024-12                25.02             3.34              65915   
2025-01                24.56             3.77              55788   
2025-02                24.34             3.69              51607   
2025-03                25.77             2.80              56233   
2025-04                25.24             2.35              36186   
2025-05           

In [131]:
# 기존 Excel 파일 확인 및 A/C Name별 분석 (A/C ON 상태만)
def check_existing_excel(filename):
    """기존 Excel 파일의 시트 구조 확인"""
    try:
        if os.path.exists(filename):
            # 기존 파일 읽기
            excel_file = pd.ExcelFile(filename)
            print(f"기존 Excel 파일 발견: {filename}")
            print(f"기존 시트 목록: {excel_file.sheet_names}")
            
            # 각 시트의 구조 확인
            for sheet_name in excel_file.sheet_names:
                df = pd.read_excel(filename, sheet_name=sheet_name)
                print(f"\n=== {sheet_name} 시트 ===")
                print(f"행 수: {len(df)}, 열 수: {len(df.columns)}")
                print(f"컬럼: {list(df.columns)}")
                if len(df) > 0:
                    print(f"샘플 데이터:")
                    print(df.head(3))
            return excel_file
        else:
            print(f"기존 Excel 파일이 없습니다: {filename}")
            return None
    except Exception as e:
        print(f"Excel 파일 확인 오류: {e}")
        return None

def analyze_by_ac_name(df, indoor_temp_col, ac_set_temp_col):
    """A/C Name별로 월별 온도 분석 (A/C ON 상태만)"""
    if 'A/C Name' not in df.columns:
        print("A/C Name 컬럼이 존재하지 않습니다.")
        return None
    
    # A/C Name별로 그룹화
    ac_names = df['A/C Name'].unique()
    print(f"발견된 A/C Name: {ac_names}")
    
    results = {}
    
    for ac_name in ac_names:
        print(f"\n=== {ac_name} 분석 중 ===")
        ac_data = df[df['A/C Name'] == ac_name].copy()
        
        if len(ac_data) == 0:
            continue
            
        # 월별 그룹화 및 통계 계산
        monthly_stats = ac_data.groupby('year_month').agg({
            indoor_temp_col: ['mean', 'std', 'count'],
            ac_set_temp_col: ['mean', 'std', 'count']
        }).round(2)
        
        # 컬럼명 정리
        monthly_stats.columns = [
            'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
            'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
        ]
        
        # 인덱스를 문자열로 변환
        monthly_stats.index = monthly_stats.index.astype(str)
        
        results[ac_name] = monthly_stats
        print(f"  - 분석 완료: {len(monthly_stats)}개 월")
    
    return results

# 기존 Excel 파일 확인
filename = "AC_setvalue_range_analysis.xlsx"
existing_excel = check_existing_excel(filename)


기존 Excel 파일이 없습니다: AC_setvalue_range_analysis.xlsx


In [132]:
# A/C Name별 분석 실행 및 Excel 업데이트 (A/C ON 상태만, 기존 포맷에 맞춰서)
if all_data is not None and 'A/C Name' in all_data.columns:
    print("\n=== A/C Name별 월별 분석 실행 (A/C ON 상태만) ===")
    
    indoor_temp_col = 'Indoor Temp.'
    ac_set_temp_col = 'A/C Set Temperature'
    
    # A/C Name별 분석
    ac_analysis_results = analyze_by_ac_name(all_data, indoor_temp_col, ac_set_temp_col)
    
    if ac_analysis_results:
        print(f"\n총 {len(ac_analysis_results)}개의 A/C Name에 대한 분석 완료")
        
        # 각 A/C Name의 분석 결과 미리보기
        for ac_name, monthly_data in ac_analysis_results.items():
            print(f"\n--- {ac_name} 분석 결과 ---")
            print(f"분석 기간: {len(monthly_data)}개 월")
            print(f"월별 평균 Indoor Temp: {monthly_data['Indoor_Temp_Mean'].mean():.2f}°C")
            print(f"월별 평균 A/C Set Temp: {monthly_data['AC_Set_Temp_Mean'].mean():.2f}°C")
        
        # A/C ON 상태 샘플 수 계산
        def calculate_ac_on_sample_counts(df):
            """A/C ON 상태의 월별 샘플 수 계산"""
            if 'A/C Name' not in df.columns:
                return None
            
            # A/C Name별 월별 샘플 수 계산
            sample_counts = df.groupby(['A/C Name', 'year_month']).size().reset_index(name='sample_count')
            
            # 월별 인덱스 생성 (1月, 2月, ..., 12月)
            months = [f"{i}月" for i in range(1, 13)]
            
            # A/C Name별 데이터를 월별로 정리
            monthly_sample_data = {}
            
            for ac_name in df['A/C Name'].unique():
                monthly_values = {}
                
                # 해당 A/C Name의 월별 샘플 수
                ac_data = sample_counts[sample_counts['A/C Name'] == ac_name]
                
                for _, row in ac_data.iterrows():
                    # period를 월로 변환 (예: 2024-07 -> 7月)
                    month_period = row['year_month']
                    month_num = int(str(month_period).split('-')[1])
                    month_name = f"{month_num}月"
                    monthly_values[month_name] = row['sample_count']
                
                monthly_sample_data[ac_name] = monthly_values
            
            # DataFrame 생성 (월별이 행, A/C Name이 열)
            df_samples = pd.DataFrame(monthly_sample_data, index=months)
            
            # 첫 번째 컬럼을 월 정보로 추가
            df_samples.insert(0, 'Unnamed: 0', months)
            
            return df_samples
        
        # A/C Fan Speed 빈도 계산
        def calculate_fan_speed_frequency(df):
            """A/C Fan Speed의 월별 빈도 계산 (A/C ON 상태만)"""
            if 'A/C Name' not in df.columns or 'A/C Fan Speed' not in df.columns:
                print("A/C Name 또는 A/C Fan Speed 컬럼이 없습니다.")
                return None
            
            # A/C ON/OFF 컬럼이 있다면 ON 상태만 필터링
            ac_onoff_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
            
            if ac_onoff_columns:
                ac_onoff_col = ac_onoff_columns[0]
                print(f"Fan Speed 빈도 계산: A/C ON/OFF 컬럼 '{ac_onoff_col}' 사용")
                
                # A/C ON 상태만 필터링
                ac_on_data = df[df[ac_onoff_col] == 'ON'].copy()
                print(f"Fan Speed 빈도 계산용 데이터: {len(ac_on_data)}개 행 (A/C ON 상태만)")
                
                if len(ac_on_data) == 0:
                    print("A/C ON 상태 데이터가 없어서 Fan Speed 빈도를 계산할 수 없습니다.")
                    return None
                
                # 필터링된 데이터 사용
                df = ac_on_data
            else:
                print("A/C ON/OFF 컬럼을 찾을 수 없어서 전체 데이터로 Fan Speed 빈도를 계산합니다.")
            
            # 월별 인덱스 생성 (1月, 2月, ..., 12月)
            months = [f"{i}月" for i in range(1, 13)]
            
            # Fan Speed 고유값 확인
            fan_speeds = df['A/C Fan Speed'].unique()
            print(f"발견된 Fan Speed 값: {fan_speeds}")
            
            # Fan Speed별 월별 빈도 계산
            fan_speed_frequency_data = []
            
            for month_name in months:
                month_num = int(month_name.replace('月', ''))
                
                for fan_speed in fan_speeds:
                    # 해당 월의 해당 Fan Speed 빈도 계산
                    month_data = df[df['month'] == month_num]
                    fan_speed_count = len(month_data[month_data['A/C Fan Speed'] == fan_speed])
                    
                    fan_speed_frequency_data.append({
                        'Unnamed: 0': month_name,
                        'Unnamed: 1': fan_speed,
                        'frequency': fan_speed_count
                    })
            
            # DataFrame 생성
            df_frequency = pd.DataFrame(fan_speed_frequency_data)
            
            # A/C Name별로 컬럼 추가
            for ac_name in df['A/C Name'].unique():
                ac_frequency_values = []
                
                for month_name in months:
                    month_num = int(month_name.replace('月', ''))
                    
                    for fan_speed in fan_speeds:
                        # 해당 A/C Name의 해당 월, 해당 Fan Speed 빈도
                        month_ac_data = df[(df['month'] == month_num) & (df['A/C Name'] == ac_name)]
                        fan_speed_count = len(month_ac_data[month_ac_data['A/C Fan Speed'] == fan_speed])
                        ac_frequency_values.append(fan_speed_count)
                
                df_frequency[ac_name] = ac_frequency_values
            
            print(f"Fan Speed 빈도 계산 완료: {len(df_frequency)}개 행")
            return df_frequency
        
        # A/C ON 상태 샘플 수 계산
        sample_counts_df = calculate_ac_on_sample_counts(all_data)
        
        # A/C Fan Speed 빈도 계산
        fan_speed_frequency_df = calculate_fan_speed_frequency(all_data)
        
        # Excel 파일 업데이트 (기존 포맷 유지)
        def update_excel_with_ac_data(excel_file, ac_analysis_results, sample_counts_df, fan_speed_frequency_df, filename):
            """기존 Excel 파일에 A/C Name별 데이터를 기존 포맷에 맞춰 입력 (A/C ON 상태만)"""
            try:
                # 기존 파일이 있는 경우 읽기
                if excel_file is not None:
                    existing_sheets = {}
                    for sheet_name in excel_file.sheet_names:
                        existing_sheets[sheet_name] = pd.read_excel(filename, sheet_name=sheet_name)
                else:
                    existing_sheets = {}
                
                # 새로운 Excel 파일 생성
                with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                    # 기존 시트들 유지하면서 업데이트
                    for sheet_name, sheet_data in existing_sheets.items():
                        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
                    
                    # A/C Name별 데이터를 기존 포맷에 맞춰 각 시트에 입력
                    if ac_analysis_results:
                        # 월별 데이터를 기존 포맷으로 변환하는 함수
                        def create_monthly_dataframe(ac_analysis_results, data_type):
                            """A/C Name별 분석 결과를 기존 포맷의 DataFrame으로 변환 (A/C ON 상태만)"""
                            # 월별 인덱스 생성 (1月, 2月, ..., 12月)
                            months = [f"{i}月" for i in range(1, 13)]
                            
                            # A/C Name별 데이터를 월별로 정리
                            monthly_data = {}
                            
                            for ac_name, monthly_stats in ac_analysis_results.items():
                                monthly_values = {}
                                
                                for month_period, row in monthly_stats.iterrows():
                                    # period를 월로 변환 (예: 2024-07 -> 7月)
                                    month_num = int(month_period.split('-')[1])
                                    month_name = f"{month_num}月"
                                    
                                    if data_type == 'indoor_mean':
                                        monthly_values[month_name] = row['Indoor_Temp_Mean']
                                    elif data_type == 'indoor_std':
                                        monthly_values[month_name] = row['Indoor_Temp_Std']
                                    elif data_type == 'ac_set_mean':
                                        monthly_values[month_name] = row['AC_Set_Temp_Mean']
                                    elif data_type == 'ac_set_std':
                                        monthly_values[month_name] = row['AC_Set_Temp_Std']
                                
                                monthly_data[ac_name] = monthly_values
                            
                            # DataFrame 생성 (월별이 행, A/C Name이 열)
                            df = pd.DataFrame(monthly_data, index=months)
                            
                            # 첫 번째 컬럼을 월 정보로 추가
                            df.insert(0, 'Unnamed: 0', months)
                            
                            return df
                        
                        # 1. Indoortemp平均 시트 (Indoor Temp 월별 평균 - A/C ON 상태만)
                        indoor_avg_df = create_monthly_dataframe(ac_analysis_results, 'indoor_mean')
                        indoor_avg_df.to_excel(writer, sheet_name='Indoortemp平均', index=False)
                        
                        # 2. 設定温度_平均値 시트 (A/C Set Temperature 월별 평균 - A/C ON 상태만)
                        ac_set_avg_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_mean')
                        ac_set_avg_df.to_excel(writer, sheet_name='設定温度_平均値', index=False)
                        
                        # 3. Indoortemp標準偏差 시트 (Indoor Temp 월별 표준편차 - A/C ON 상태만)
                        indoor_std_df = create_monthly_dataframe(ac_analysis_results, 'indoor_std')
                        indoor_std_df.to_excel(writer, sheet_name='Indoortemp標準偏差', index=False)
                        
                        # 4. 設定温度_標準偏差 시트 (A/C Set Temperature 월별 표준편차 - A/C ON 상태만)
                        ac_set_std_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_std')
                        ac_set_std_df.to_excel(writer, sheet_name='設定温度_標準偏差', index=False)
                    
                    # 5. 室内機別_サンプル数 시트 (A/C ON 상태 샘플 수)
                    if sample_counts_df is not None:
                        sample_counts_df.to_excel(writer, sheet_name='室内機別_サンプル数', index=False)
                        print("室内機別_サンプル数 시트에 A/C ON 상태 샘플 수 입력 완료")
                    
                    # 6. FanSpeed頻度 시트 (A/C Fan Speed 빈도)
                    if fan_speed_frequency_df is not None:
                        fan_speed_frequency_df.to_excel(writer, sheet_name='FanSpeed頻度', index=False)
                        print("FanSpeed頻度 시트에 A/C Fan Speed 빈도 입력 완료")
                
                print(f"Excel 파일 업데이트 완료: {filename}")
                return True
                
            except Exception as e:
                print(f"Excel 파일 업데이트 오류: {e}")
                return False
        
        # Excel 파일 업데이트 실행
        print("\n=== Excel 파일 업데이트 실행 (A/C ON 상태만) ===")
        success = update_excel_with_ac_data(existing_excel, ac_analysis_results, sample_counts_df, fan_speed_frequency_df, filename)
        
        if success:
            print(f"\n=== 업데이트된 시트 목록 (A/C ON 상태만) ===")
            print("1. Indoortemp平均 - Indoor Temp 월별 평균 (A/C ON 상태만)")
            print("2. 設定温度_平均値 - A/C Set Temperature 월별 평균 (A/C ON 상태만)") 
            print("3. Indoortemp標準偏差 - Indoor Temp 월별 표준편차 (A/C ON 상태만)")
            print("4. 設定温度_標準偏差 - A/C Set Temperature 월별 표준편차 (A/C ON 상태만)")
            print("5. 室内機別_サンプル数 - A/C ON 상태 월별 샘플 수")
            print("6. FanSpeed頻度 - A/C Fan Speed 월별 빈도")
            
            # 업데이트된 파일 확인
            updated_excel = pd.ExcelFile(filename)
            print(f"\n최종 시트 목록: {updated_excel.sheet_names}")
        else:
            print("Excel 파일 업데이트에 실패했습니다.")
    else:
        print("A/C Name별 분석을 수행할 수 없습니다.")
else:
    print("A/C Name 컬럼이 없거나 데이터가 없습니다.")
    if all_data is not None:
        print(f"사용 가능한 컬럼: {list(all_data.columns)}")



=== A/C Name별 월별 분석 실행 (A/C ON 상태만) ===
발견된 A/C Name: ['G-22' 'G-21' 'G-24' 'G-23' 'D-1南1' 'D-3南2' 'D-6北1' 'D-5南1' 'A-26' 'E-17'
 'E-11南3' 'E-9南1' 'E-14北2' 'E-13北1' 'E-10南2' 'F-18' 'D-4北2' 'E-15北3'
 'D-7南2' 'D-8北2' 'E-16北4' 'A-25' 'E-12南4' 'D-2北1' 'F-19' 'F-20']

=== G-22 분석 중 ===
  - 분석 완료: 15개 월

=== G-21 분석 중 ===
  - 분석 완료: 15개 월

=== G-24 분석 중 ===
  - 분석 완료: 15개 월

=== G-23 분석 중 ===
  - 분석 완료: 7개 월

=== D-1南1 분석 중 ===
  - 분석 완료: 15개 월

=== D-3南2 분석 중 ===
  - 분석 완료: 15개 월

=== D-6北1 분석 중 ===
  - 분석 완료: 15개 월

=== D-5南1 분석 중 ===
  - 분석 완료: 15개 월

=== A-26 분석 중 ===
  - 분석 완료: 15개 월

=== E-17 분석 중 ===
  - 분석 완료: 15개 월

=== E-11南3 분석 중 ===
  - 분석 완료: 15개 월

=== E-9南1 분석 중 ===
  - 분석 완료: 15개 월

=== E-14北2 분석 중 ===
  - 분석 완료: 15개 월

=== E-13北1 분석 중 ===
  - 분석 완료: 15개 월

=== E-10南2 분석 중 ===
  - 분석 완료: 15개 월

=== F-18 분석 중 ===
  - 분석 완료: 15개 월

=== D-4北2 분석 중 ===
  - 분석 완료: 14개 월

=== E-15北3 분석 중 ===
  - 분석 완료: 15개 월

=== D-7南2 분석 중 ===
  - 분석 완료: 14개 월

=== D-8北2 분석 중 ===
  - 분석 완료: 13개 월

In [133]:
# 기존 Excel 파일의 시트 구조 상세 확인 (FanSpeed頻度 시트 포함)
filename = "AC_setvalue_range_analysis.xlsx"

if os.path.exists(filename):
    print(f"=== {filename} 파일 구조 분석 ===")
    
    # Excel 파일 읽기
    excel_file = pd.ExcelFile(filename)
    print(f"시트 목록: {excel_file.sheet_names}")
    
    # FanSpeed頻度 시트가 있는지 확인
    fanspeed_sheet_name = None
    for sheet_name in excel_file.sheet_names:
        if 'FanSpeed' in sheet_name or '頻度' in sheet_name:
            fanspeed_sheet_name = sheet_name
            break
    
    if fanspeed_sheet_name:
        print(f"\n{'='*60}")
        print(f"발견된 FanSpeed頻度 시트: {fanspeed_sheet_name}")
        print(f"{'='*60}")
        
        df = pd.read_excel(filename, sheet_name=fanspeed_sheet_name)
        print(f"행 수: {len(df)}, 열 수: {len(df.columns)}")
        print(f"인덱스 타입: {type(df.index)}")
        print(f"인덱스 값: {list(df.index)}")
        print(f"컬럼명: {list(df.columns)}")
        
        if len(df) > 0:
            print(f"\n전체 데이터:")
            print(df)
            
            # 각 컬럼의 데이터 타입 확인
            print(f"\n데이터 타입:")
            print(df.dtypes)
            
            # NaN 값 확인
            print(f"\nNaN 값 개수:")
            print(df.isnull().sum())
            
            # 첫 번째 컬럼의 고유값 확인 (월 정보)
            if 'Unnamed: 0' in df.columns:
                print(f"\n첫 번째 컬럼(월) 고유값:")
                print(df['Unnamed: 0'].unique())
            
            # 두 번째 컬럼의 고유값 확인 (Fan Speed 정보)
            if 'Unnamed: 1' in df.columns:
                print(f"\n두 번째 컬럼(Fan Speed) 고유값:")
                print(df['Unnamed: 1'].unique())
        else:
            print("데이터가 비어있습니다.")
    else:
        print("FanSpeed頻度 관련 시트를 찾을 수 없습니다.")
    
    # 다른 시트들도 간단히 확인
    for sheet_name in excel_file.sheet_names:
        if sheet_name != fanspeed_sheet_name:
            print(f"\n{'='*30}")
            print(f"시트명: {sheet_name}")
            print(f"{'='*30}")
            
            df = pd.read_excel(filename, sheet_name=sheet_name)
            print(f"행 수: {len(df)}, 열 수: {len(df.columns)}")
            print(f"컬럼명: {list(df.columns)}")
            
            if len(df) > 0:
                print(f"\n샘플 데이터:")
                print(df.head(3))
else:
    print(f"파일이 존재하지 않습니다: {filename}")


=== AC_setvalue_range_analysis.xlsx 파일 구조 분석 ===
시트 목록: ['Indoortemp平均', '設定温度_平均値', 'Indoortemp標準偏差', '設定温度_標準偏差', '室内機別_サンプル数', 'FanSpeed頻度']

발견된 FanSpeed頻度 시트: FanSpeed頻度
행 수: 60, 열 수: 29
인덱스 타입: <class 'pandas.core.indexes.range.RangeIndex'>
인덱스 값: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
컬럼명: ['Unnamed: 0', 'Unnamed: 1', 'frequency', 'G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']

전체 데이터:
   Unnamed: 0 Unnamed: 1  frequency  G-22  G-21  G-24  G-23  D-1南1  D-3南2  \
0          1月     Medium      19630  1286  1626    50     0   6642   6642   
1          1月        Low      35902     0     0     0     0   2206   2206 

In [134]:
# 두 건물 데이터 처리 및 분석
def process_building_analysis(building_name, data_path):
    """특정 건물의 데이터를 로드하고 분석"""
    print(f"\n{'='*60}")
    print(f"=== {building_name} 건물 데이터 처리 ===")
    print(f"{'='*60}")
    
    # 데이터 로드
    csv_files = glob.glob(os.path.join(data_path, "*.csv"))
    print(f"{building_name}: 총 {len(csv_files)}개의 CSV 파일 발견")
    
    all_data = []
    for i, file_path in enumerate(csv_files):
        print(f"로딩 중... ({i+1}/{len(csv_files)}) {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"  오류: {e}")
            continue
    
    if not all_data:
        print(f"{building_name} 데이터 로딩 실패")
        return None
    
    # 데이터 통합
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"{building_name} 통합 완료: {len(combined_df)}개 행")
    
    # 날짜 컬럼 처리
    time_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    if time_columns:
        date_column = time_columns[0]
        print(f"{building_name} 날짜 컬럼 '{date_column}' 사용")
        combined_df[date_column] = pd.to_datetime(combined_df[date_column], errors='coerce')
        combined_df['year_month'] = combined_df[date_column].dt.to_period('M')
        combined_df['year'] = combined_df[date_column].dt.year
        combined_df['month'] = combined_df[date_column].dt.month
    
    # A/C ON/OFF 필터링
    ac_onoff_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
    if ac_onoff_columns:
        ac_onoff_col = ac_onoff_columns[0]
        print(f"{building_name} A/C ON/OFF 컬럼 '{ac_onoff_col}' 사용")
        
        total_rows = len(combined_df)
        ac_on_data = combined_df[combined_df[ac_onoff_col] == 'ON'].copy()
        ac_on_rows = len(ac_on_data)
        print(f"{building_name} A/C ON 상태 데이터: {ac_on_rows}개 행 ({ac_on_rows/total_rows*100:.1f}%)")
        
        if ac_on_rows > 0:
            return ac_on_data
        else:
            print(f"{building_name} A/C ON 상태 데이터가 없습니다.")
            return None
    else:
        print(f"{building_name} A/C ON/OFF 컬럼을 찾을 수 없습니다.")
        return combined_df

# 두 건물 데이터 처리
buildings = {
    'Clea': '../../Clea/ac-control/',
    'IsetanMitsukoshi': '../../IsetanMitsukoshi/ac-control/'
}

building_data = {}

for building_name, data_path in buildings.items():
    print(f"\n{building_name} 건물 데이터 처리 시작...")
    data = process_building_analysis(building_name, data_path)
    if data is not None:
        building_data[building_name] = data
        print(f"{building_name} 데이터 처리 완료: {len(data)}개 행")
    else:
        print(f"{building_name} 데이터 처리 실패")

print(f"\n총 {len(building_data)}개 건물 데이터 처리 완료")



Clea 건물 데이터 처리 시작...

=== Clea 건물 데이터 처리 ===
Clea: 총 30개의 CSV 파일 발견
로딩 중... (1/30) ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
로딩 중... (2/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
로딩 중... (3/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
로딩 중... (4/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
로딩 중... (5/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
로딩 중... (6/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv
로딩 중... (7/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv
로딩 중... (8/30) ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv
로딩 중... (9/30) ac-control-50,66,67,65,59,63,57,68

In [135]:
# 각 건물별 분석 및 Excel 파일 생성
def analyze_by_ac_name(df, indoor_temp_col, ac_set_temp_col):
    """A/C Name별 월별 온도 분석 (A/C ON 상태만)"""
    if 'A/C Name' not in df.columns:
        print("A/C Name 컬럼이 없습니다.")
        return None
    
    ac_names = df['A/C Name'].unique()
    print(f"총 {len(ac_names)}개의 A/C Name 발견")
    
    ac_analysis_results = {}
    
    for ac_name in ac_names:
        print(f"\n=== {ac_name} 분석 중 ===")
        
        # 해당 A/C Name의 데이터만 필터링
        ac_data = df[df['A/C Name'] == ac_name].copy()
        
        if len(ac_data) == 0:
            print(f"  {ac_name} 데이터가 없습니다.")
            continue
        
        # 월별 그룹화 및 통계 계산
        monthly_stats = ac_data.groupby('year_month').agg({
            indoor_temp_col: ['mean', 'std', 'count'],
            ac_set_temp_col: ['mean', 'std', 'count']
        }).round(2)
        
        # 컬럼명 정리
        monthly_stats.columns = [
            'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
            'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
        ]
        
        # 인덱스를 문자열로 변환
        monthly_stats.index = monthly_stats.index.astype(str)
        
        ac_analysis_results[ac_name] = monthly_stats
        print(f"  분석 완료: {len(monthly_stats)}개 월")
    
    return ac_analysis_results

def calculate_ac_on_sample_counts(df):
    """A/C ON 상태의 월별 샘플 수 계산"""
    if 'A/C Name' not in df.columns:
        return None
    
    # A/C Name별 월별 샘플 수 계산
    sample_counts = df.groupby(['A/C Name', 'year_month']).size().reset_index(name='sample_count')
    
    # 월별 인덱스 생성 (1月, 2月, ..., 12月)
    months = [f"{i}月" for i in range(1, 13)]
    
    # A/C Name별 데이터를 월별로 정리
    monthly_sample_data = {}
    
    for ac_name in df['A/C Name'].unique():
        monthly_values = {}
        
        # 해당 A/C Name의 월별 샘플 수
        ac_data = sample_counts[sample_counts['A/C Name'] == ac_name]
        
        for _, row in ac_data.iterrows():
            # period를 월로 변환 (예: 2024-07 -> 7月)
            month_period = row['year_month']
            month_num = int(str(month_period).split('-')[1])
            month_name = f"{month_num}月"
            monthly_values[month_name] = row['sample_count']
        
        monthly_sample_data[ac_name] = monthly_values
    
    # DataFrame 생성 (월별이 행, A/C Name이 열)
    df_samples = pd.DataFrame(monthly_sample_data, index=months)
    
    # 첫 번째 컬럼을 월 정보로 추가
    df_samples.insert(0, 'Unnamed: 0', months)
    
    return df_samples

def calculate_fan_speed_frequency(df):
    """A/C Fan Speed의 월별 빈도 계산 (A/C ON 상태만)"""
    if 'A/C Name' not in df.columns or 'A/C Fan Speed' not in df.columns:
        print("A/C Name 또는 A/C Fan Speed 컬럼이 없습니다.")
        return None
    
    # 월별 인덱스 생성 (1月, 2月, ..., 12月)
    months = [f"{i}月" for i in range(1, 13)]
    
    # Fan Speed 고유값 확인
    fan_speeds = df['A/C Fan Speed'].unique()
    print(f"발견된 Fan Speed 값: {fan_speeds}")
    
    # Fan Speed별 월별 빈도 계산
    fan_speed_frequency_data = []
    
    for month_name in months:
        month_num = int(month_name.replace('月', ''))
        
        for fan_speed in fan_speeds:
            # 해당 월의 해당 Fan Speed 빈도 계산
            month_data = df[df['month'] == month_num]
            fan_speed_count = len(month_data[month_data['A/C Fan Speed'] == fan_speed])
            
            fan_speed_frequency_data.append({
                'Unnamed: 0': month_name,
                'Unnamed: 1': fan_speed,
                'frequency': fan_speed_count
            })
    
    # DataFrame 생성
    df_frequency = pd.DataFrame(fan_speed_frequency_data)
    
    # A/C Name별로 컬럼 추가
    for ac_name in df['A/C Name'].unique():
        ac_frequency_values = []
        
        for month_name in months:
            month_num = int(month_name.replace('月', ''))
            
            for fan_speed in fan_speeds:
                # 해당 A/C Name의 해당 월, 해당 Fan Speed 빈도
                month_ac_data = df[(df['month'] == month_num) & (df['A/C Name'] == ac_name)]
                fan_speed_count = len(month_ac_data[month_ac_data['A/C Fan Speed'] == fan_speed])
                ac_frequency_values.append(fan_speed_count)
        
        df_frequency[ac_name] = ac_frequency_values
    
    print(f"Fan Speed 빈도 계산 완료: {len(df_frequency)}개 행")
    return df_frequency

def create_monthly_dataframe(ac_analysis_results, data_type):
    """A/C Name별 분석 결과를 기존 포맷의 DataFrame으로 변환 (A/C ON 상태만)"""
    # 월별 인덱스 생성 (1月, 2月, ..., 12月)
    months = [f"{i}月" for i in range(1, 13)]
    
    # A/C Name별 데이터를 월별로 정리
    monthly_data = {}
    
    for ac_name, monthly_stats in ac_analysis_results.items():
        monthly_values = {}
        
        for month_period, row in monthly_stats.iterrows():
            # period를 월로 변환 (예: 2024-07 -> 7月)
            month_num = int(month_period.split('-')[1])
            month_name = f"{month_num}月"
            
            if data_type == 'indoor_mean':
                monthly_values[month_name] = row['Indoor_Temp_Mean']
            elif data_type == 'indoor_std':
                monthly_values[month_name] = row['Indoor_Temp_Std']
            elif data_type == 'ac_set_mean':
                monthly_values[month_name] = row['AC_Set_Temp_Mean']
            elif data_type == 'ac_set_std':
                monthly_values[month_name] = row['AC_Set_Temp_Std']
        
        monthly_data[ac_name] = monthly_values
    
    # DataFrame 생성 (월별이 행, A/C Name이 열)
    df = pd.DataFrame(monthly_data, index=months)
    
    # 첫 번째 컬럼을 월 정보로 추가
    df.insert(0, 'Unnamed: 0', months)
    
    return df

def update_excel_with_building_data(building_name, ac_analysis_results, sample_counts_df, fan_speed_frequency_df):
    """건물별 Excel 파일 생성"""
    filename = f"AC_setvalue_range_analysis_{building_name}.xlsx"
    
    try:
        # 새로운 Excel 파일 생성
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # A/C Name별 데이터를 기존 포맷에 맞춰 각 시트에 입력
            if ac_analysis_results:
                # 1. Indoortemp平均 시트 (Indoor Temp 월별 평균 - A/C ON 상태만)
                indoor_avg_df = create_monthly_dataframe(ac_analysis_results, 'indoor_mean')
                indoor_avg_df.to_excel(writer, sheet_name='Indoortemp平均', index=False)
                
                # 2. 設定温度_平均値 시트 (A/C Set Temperature 월별 평균 - A/C ON 상태만)
                ac_set_avg_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_mean')
                ac_set_avg_df.to_excel(writer, sheet_name='設定温度_平均値', index=False)
                
                # 3. Indoortemp標準偏差 시트 (Indoor Temp 월별 표준편차 - A/C ON 상태만)
                indoor_std_df = create_monthly_dataframe(ac_analysis_results, 'indoor_std')
                indoor_std_df.to_excel(writer, sheet_name='Indoortemp標準偏差', index=False)
                
                # 4. 設定温度_標準偏差 시트 (A/C Set Temperature 월별 표준편차 - A/C ON 상태만)
                ac_set_std_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_std')
                ac_set_std_df.to_excel(writer, sheet_name='設定温度_標準偏差', index=False)
            
            # 5. 室内機別_サンプル数 시트 (A/C ON 상태 샘플 수)
            if sample_counts_df is not None:
                sample_counts_df.to_excel(writer, sheet_name='室内機別_サンプル数', index=False)
                print(f"{building_name}: 室内機別_サンプル数 시트에 A/C ON 상태 샘플 수 입력 완료")
            
            # 6. FanSpeed頻度 시트 (A/C Fan Speed 빈도)
            if fan_speed_frequency_df is not None:
                fan_speed_frequency_df.to_excel(writer, sheet_name='FanSpeed頻度', index=False)
                print(f"{building_name}: FanSpeed頻度 시트에 A/C Fan Speed 빈도 입력 완료")
        
        print(f"{building_name}: Excel 파일 생성 완료 - {filename}")
        return True
        
    except Exception as e:
        print(f"{building_name}: Excel 파일 생성 오류 - {e}")
        return False

# 각 건물별 분석 및 Excel 파일 생성
for building_name, data in building_data.items():
    print(f"\n{'='*60}")
    print(f"=== {building_name} 건물 분석 및 Excel 파일 생성 ===")
    print(f"{'='*60}")
    
    if 'A/C Name' in data.columns:
        indoor_temp_col = 'Indoor Temp.'
        ac_set_temp_col = 'A/C Set Temperature'
        
        # A/C Name별 분석
        ac_analysis_results = analyze_by_ac_name(data, indoor_temp_col, ac_set_temp_col)
        
        if ac_analysis_results:
            print(f"{building_name}: 총 {len(ac_analysis_results)}개의 A/C Name에 대한 분석 완료")
            
            # A/C ON 상태 샘플 수 계산
            sample_counts_df = calculate_ac_on_sample_counts(data)
            
            # A/C Fan Speed 빈도 계산
            fan_speed_frequency_df = calculate_fan_speed_frequency(data)
            
            # Excel 파일 생성
            success = update_excel_with_building_data(building_name, ac_analysis_results, sample_counts_df, fan_speed_frequency_df)
            
            if success:
                print(f"\n=== {building_name} 업데이트된 시트 목록 (A/C ON 상태만) ===")
                print("1. Indoortemp平均 - Indoor Temp 월별 평균 (A/C ON 상태만)")
                print("2. 設定温度_平均値 - A/C Set Temperature 월별 평균 (A/C ON 상태만)") 
                print("3. Indoortemp標準偏差 - Indoor Temp 월별 표준편차 (A/C ON 상태만)")
                print("4. 設定温度_標準偏差 - A/C Set Temperature 월별 표준편차 (A/C ON 상태만)")
                print("5. 室内機別_サンプル数 - A/C ON 상태 월별 샘플 수")
                print("6. FanSpeed頻度 - A/C Fan Speed 월별 빈도")
            else:
                print(f"{building_name}: Excel 파일 생성에 실패했습니다.")
        else:
            print(f"{building_name}: A/C Name별 분석을 수행할 수 없습니다.")
    else:
        print(f"{building_name}: A/C Name 컬럼이 없습니다.")
        print(f"사용 가능한 컬럼: {list(data.columns)}")

print(f"\n{'='*60}")
print("=== 모든 건물 분석 완료 ===")
print(f"{'='*60}")



=== Clea 건물 분석 및 Excel 파일 생성 ===
총 26개의 A/C Name 발견

=== G-22 분석 중 ===
  분석 완료: 15개 월

=== G-21 분석 중 ===
  분석 완료: 15개 월

=== G-24 분석 중 ===
  분석 완료: 15개 월

=== G-23 분석 중 ===
  분석 완료: 7개 월

=== D-1南1 분석 중 ===
  분석 완료: 15개 월

=== D-3南2 분석 중 ===
  분석 완료: 15개 월

=== D-6北1 분석 중 ===
  분석 완료: 15개 월

=== D-5南1 분석 중 ===
  분석 완료: 15개 월

=== A-26 분석 중 ===
  분석 완료: 15개 월

=== E-17 분석 중 ===
  분석 완료: 15개 월

=== E-11南3 분석 중 ===
  분석 완료: 15개 월

=== E-9南1 분석 중 ===
  분석 완료: 15개 월

=== E-14北2 분석 중 ===
  분석 완료: 15개 월

=== E-13北1 분석 중 ===
  분석 완료: 15개 월

=== E-10南2 분석 중 ===
  분석 완료: 15개 월

=== F-18 분석 중 ===
  분석 완료: 15개 월

=== D-4北2 분석 중 ===
  분석 완료: 14개 월

=== E-15北3 분석 중 ===
  분석 완료: 15개 월

=== D-7南2 분석 중 ===
  분석 완료: 14개 월

=== D-8北2 분석 중 ===
  분석 완료: 13개 월

=== E-16北4 분석 중 ===
  분석 완료: 15개 월

=== A-25 분석 중 ===
  분석 완료: 15개 월

=== E-12南4 분석 중 ===
  분석 완료: 15개 월

=== D-2北1 분석 중 ===
  분석 완료: 12개 월

=== F-19 분석 중 ===
  분석 완료: 15개 월

=== F-20 분석 중 ===
  분석 완료: 15개 월
Clea: 총 26개의 A/C Name에 대한 분석 완료
발견된 Fan S